In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from yt_dlp import YoutubeDL
from datetime import datetime
import json
import pandas as pd 
import numpy as np
from berean_transcripts.utils import data_dir

ffmpeg -i G7v8KAk6kLk.mkv -c:v copy -c:a aac G7v8KAk6kLk.mp4


In [3]:
import cv2

In [4]:
video_file =  str(data_dir / 'video' / 'G7v8KAk6kLk.mp4')

cap = cv2.VideoCapture(video_file)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

ret, prev_frame = cap.read()

In [38]:
import cv2
import logging
import numpy as np

def frame_difference(prev_frame, curr_frame):
    grayA = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(grayA, grayB)
    return np.sum(diff)

def extract_sections(video_file, 
                     threshold=3_000_000, 
                     min_seconds=2, 
                     frame_skip=30, 
                     start_time=None, 
                     end_time=None):
    logging.info("Identifying sections from sermon video")
    cap = cv2.VideoCapture(video_file)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Frame rate of the video is: {fps} FPS")

    # Convert start_time and end_time to start_frame and end_frame
    start_frame = int(start_time * fps) if start_time is not None else 0
    end_frame = int(end_time * fps) if end_time is not None else frame_count
    
    # Set the initial position if start_time is provided
    if start_time is not None:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    ret, prev_frame = cap.read()
    section_transitions = [(start_time or 0, start_frame)]

    for i in range(start_frame, end_frame, frame_skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, curr_frame = cap.read()
        if not ret:
            break

        diff = frame_difference(prev_frame, curr_frame)
        timestamp = i / fps

        if diff > threshold and (timestamp - section_transitions[-1][0]) > min_seconds:
            section_transitions.append((timestamp, i))

        prev_frame = curr_frame

    cap.release()

    sections = []
    for start, end in zip(section_transitions[:-1], section_transitions[1:]):
        sections.append((start[1]/fps, end[1]/fps))

    sections.append((section_transitions[-1][1]/fps, end_frame/fps))
    sermon_section = max(sections, key=lambda x: x[1] - x[0])

    logging.info(f"Identified {len(sections)} sections, Sermon section is between frames {sermon_section[0]} and {sermon_section[1]}")
    
    return sections, sermon_section


In [39]:
25*60, 33*60

(1500, 1980)

In [49]:
# To use the function, simply call:
# sections, sermon_section = extract_sections(video_file, 
#                                             threshold=5_000_000)

# Example usage:
sections, sermon_section = extract_sections(video_file, 
                                            threshold=75_000_000,
                                            start_time=25*60, end_time=50*60)
print(f"Sections: {sections}")
print(f"Sermon Section: {sermon_section}")

Frame rate of the video is: 30 FPS
Sections: [(1500.0, 1555.0), (1555.0, 1561.0), (1561.0, 1795.0), (1795.0, 1845.0), (1845.0, 3000.0)]
Sermon Section: (1845.0, 3000.0)


In [44]:
45*60

2700

In [35]:
# baptism
(1561.0 / 60, 1794.0 / 60)

(26.016666666666666, 29.9)

In [41]:
# transition between baptism and sermon
1795.0/60

29.916666666666668

In [42]:
# sermon
1845.0/60

30.75

In [47]:
45*60

2700

In [50]:
sections, sermon_section = extract_sections(video_file, 
                                            threshold=75_000_000,
                                            start_time=25*60, end_time=80*60)
print(f"Sections: {sections}")
print(f"Sermon Section: {sermon_section}")

Frame rate of the video is: 30 FPS
Sections: [(1500.0, 1555.0), (1555.0, 1561.0), (1561.0, 1795.0), (1795.0, 1845.0), (1845.0, 4049.0), (4049.0, 4067.0), (4067.0, 4472.0), (4472.0, 4708.0), (4708.0, 4730.0), (4730.0, 4800.0)]
Sermon Section: (1845.0, 4049.0)


In [51]:
# 75_000_000 threshold is better
sections, sermon_section = extract_sections(video_file, 
                                            threshold=75_000_000,)
print(f"Sections: {sections}")
print(f"Sermon Section: {sermon_section}")

Frame rate of the video is: 30 FPS
Sections: [(0.0, 409.0), (409.0, 683.0), (683.0, 720.0), (720.0, 743.0), (743.0, 753.0), (753.0, 760.0), (760.0, 786.0), (786.0, 877.0), (877.0, 925.0), (925.0, 983.0), (983.0, 1555.0), (1555.0, 1561.0), (1561.0, 1795.0), (1795.0, 1845.0), (1845.0, 4049.0), (4049.0, 4067.0), (4067.0, 4472.0), (4472.0, 4708.0), (4708.0, 4730.0), (4730.0, 4822.0), (4822.0, 4825.0), (4825.0, 4841.166666666667)]
Sermon Section: (1845.0, 4049.0)


In [52]:
sections, sermon_section = extract_sections(video_file, 
                                            threshold=85_000_000,)
print(f"Sections: {sections}")
print(f"Sermon Section: {sermon_section}")

Frame rate of the video is: 30 FPS
Sections: [(0.0, 409.0), (409.0, 720.0), (720.0, 743.0), (743.0, 753.0), (753.0, 760.0), (760.0, 786.0), (786.0, 877.0), (877.0, 925.0), (925.0, 983.0), (983.0, 1555.0), (1555.0, 1561.0), (1561.0, 1795.0), (1795.0, 4049.0), (4049.0, 4067.0), (4067.0, 4472.0), (4472.0, 4708.0), (4708.0, 4730.0), (4730.0, 4822.0), (4822.0, 4825.0), (4825.0, 4841.166666666667)]
Sermon Section: (1795.0, 4049.0)


In [54]:
1795/60, 1845/60

(29.916666666666668, 30.75)

In [14]:
sections

[(0, 12240),
 (12240, 12330),
 (12330, 12420),
 (12420, 12510),
 (12510, 12600),
 (12600, 12690),
 (12690, 12780),
 (12780, 12870),
 (12870, 12960),
 (12960, 13050),
 (13050, 13140),
 (13140, 13230),
 (13230, 13320),
 (13320, 13410),
 (13410, 13500),
 (13500, 13590),
 (13590, 13680),
 (13680, 13770),
 (13770, 13860),
 (13860, 13950),
 (13950, 14040),
 (14040, 14130),
 (14130, 14220),
 (14220, 14310),
 (14310, 14400),
 (14400, 14490),
 (14490, 14580),
 (14580, 14670),
 (14670, 14760),
 (14760, 14850),
 (14850, 14940),
 (14940, 15030),
 (15030, 15120),
 (15120, 15210),
 (15210, 15300),
 (15300, 15390),
 (15390, 15480),
 (15480, 15570),
 (15570, 15660),
 (15660, 15750),
 (15750, 15840),
 (15840, 15930),
 (15930, 16020),
 (16020, 16110),
 (16110, 16200),
 (16200, 16290),
 (16290, 16380),
 (16380, 16470),
 (16470, 16560),
 (16560, 16650),
 (16650, 16740),
 (16740, 16830),
 (16830, 16920),
 (16920, 17010),
 (17010, 17100),
 (17100, 17190),
 (17190, 17280),
 (17280, 17370),
 (17370, 17460),
 

In [3]:
# video preprocessing
import cv2

video_path =  str(data_dir / 'video' / 'G7v8KAk6kLk.mp4')
cap = cv2.VideoCapture(video_path)
frames = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)


: 

In [ ]:
# calc frame differences
frame_diffs = []
for i in range(1, len(frames)):
    grayA = cv2.cvtColor(frames[i - 1], cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(grayA, grayB)
    frame_diffs.append(diff)

In [ ]:
#  thresholding
threshold = 1000000  # this value will depend on your specific video and may need tuning
significant_changes = []

for i, diff in enumerate(frame_diffs):
    sum_diff = np.sum(diff)
    if sum_diff > threshold:
        significant_changes.append(i)

In [ ]:
# section identifcation
sections = []
start = 0
for end in significant_changes:
    sections.append((start, end))
    start = end
sections.append((start, len(frames) - 1))  # till the last frame


In [ ]:
# sermon identification
sermon_section = max(sections, key=lambda x: x[1] - x[0])


In [18]:
cache_file = data_dir / "video_details_cache.json"
try:
    with open(cache_file, "r") as f:
        video_details_cache = json.load(f)
except FileNotFoundError:
    video_details_cache = {}


In [4]:
ydl_opts = {"quiet": True}
video_id = "G7v8KAk6kLk"
with YoutubeDL(ydl_opts) as ydl:
    # info_dict = ydl.extract_info(
    #     f"https://www.youtube.com/watch?v={video_id}", download=False
    # )
    # video_details_cache[video_id] = info_dict
    info_dict = ydl.extract_info(
        f"https://www.youtube.com/watch?v={video_id}", download=False
    )

In [19]:
video_details_cache['G7v8KAk6kLk']

{'id': 'G7v8KAk6kLk',
 'title': 'Berean Community Church Sunday Service 10.15.23',
 'formats': [{'format_id': 'sb2',
   'format_note': 'storyboard',
   'ext': 'mhtml',
   'protocol': 'mhtml',
   'acodec': 'none',
   'vcodec': 'none',
   'url': 'https://i.ytimg.com/sb/G7v8KAk6kLk/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjR0rOpBg==&sigh=rs$AOn4CLCujxh8gp85IQUeFtWrlVHlM8ET4A',
   'width': 48,
   'height': 27,
   'fps': 0.02065688907250568,
   'rows': 10,
   'columns': 10,
   'fragments': [{'url': 'https://i.ytimg.com/sb/G7v8KAk6kLk/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjR0rOpBg==&sigh=rs$AOn4CLCujxh8gp85IQUeFtWrlVHlM8ET4A',
     'duration': 4841.0}],
   'resolution': '48x27',
   'aspect_ratio': 1.78,
   'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4556.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accep

In [45]:
# Initialize list to store flattened dictionaries
flattened_data = []

# Flatten dictionary structure
for video, attributes in video_details_cache.items():
    # print(video)
    id_ = attributes['id']
    title = attributes['title']
    upload_date = attributes['upload_date']
    upload_date_formatted = datetime.strptime(
                upload_date, "%Y%m%d"
            ).strftime(
                "%Y-%m-%d"
            )
    flattened_dict = {'id': id_, 
                      'title': title,
                       'upload_date': upload_date_formatted
                      }
    flattened_data.append(flattened_dict)

df = pd.DataFrame(flattened_data)


In [48]:
df = df.sort_values(['upload_date'], ascending=False)

In [49]:
df.head()

,id,title,upload_date
315,G7v8KAk6kLk,Berean Community Church Sunday Service 10.15.23,2023-10-16
311,6_z7k2JUwew,Berean Community Church Sunday Service 10.8.2023,2023-10-09
312,GTKTFjQReno,Berean Community Church Wednesday Bible Study ...,2023-10-08
313,TJqmuLfW3qE,Berean Community Church Wednesday Bible Study ...,2023-10-05
314,gk4dCXqufCY,Berean Community Church - Sunday Service 10/1,2023-10-02


In [44]:
info_dict

{'id': 'G7v8KAk6kLk',
 'title': 'Berean Community Church Sunday Service 10.15.23',
 'formats': [{'format_id': 'sb2',
   'format_note': 'storyboard',
   'ext': 'mhtml',
   'protocol': 'mhtml',
   'acodec': 'none',
   'vcodec': 'none',
   'url': 'https://i.ytimg.com/sb/G7v8KAk6kLk/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjR0rOpBg==&sigh=rs$AOn4CLCujxh8gp85IQUeFtWrlVHlM8ET4A',
   'width': 48,
   'height': 27,
   'fps': 0.02065688907250568,
   'rows': 10,
   'columns': 10,
   'fragments': [{'url': 'https://i.ytimg.com/sb/G7v8KAk6kLk/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjR0rOpBg==&sigh=rs$AOn4CLCujxh8gp85IQUeFtWrlVHlM8ET4A',
     'duration': 4841.0}],
   'resolution': '48x27',
   'aspect_ratio': 1.78,
   'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.24 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Langu

In [12]:
info_dict['id']

'G7v8KAk6kLk'

In [6]:
info_dict['title']

'Berean Community Church Sunday Service 10.15.23'

In [7]:
info_dict['upload_date']

'20231016'